# `WHEN Frequency for Activity AU Description`

### `Created/Modified By: Z Liu`
 
* `1, Rewrite architecture of the code`
* `2, Fix a warning "a value is trying to be set on a copy"; using df.at[] or df.loc[] instead of df[][]`
* `3, Modify code so as to capture adhoc & ad hoc` 
* `4, Add similarity attribute to compare predictions and ground truth`
* `5, Modify code so as to capture WHEN containing conjunction such as annual and quarterly `
* `6, Remove stop words on predictions and ground truth before comparing similarity`
* `7, Modify code so as to capture WHEN containing ... as ... basis (monthly as needed basis)`
* `8, Be able to capture bi-..., bi ..., bi... etc`
* `9, Solve error, "float is not subscriptable", desc = str(desc)`
* `10, Add confidence, starting index, ending index`

In [1]:
%%time

import os
import re
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
import torch.nn.functional as F

# Deep Learning Libraries
import spacy
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer

from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModel, AutoModelForQuestionAnswering,pipeline

import nltk
# nltk.data.path.append('/home/mca_fix/share/nltk_data/') 
nltk.data.path.append('/home/mcafixmlpython/lib/nltk_data/') 
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()


# nlp = spacy.load("/home/mca_fix/share/en_core_web_md-3.0.0/en_core_web_md/en_core_web_md-3.0.0")
nlp = spacy.load("/home/mcafixmlpython/lib/en_core_web_md-3.0.0/en_core_web_md/en_core_web_md-3.0.0")

# Loading BERT-BASE-NLI-MEAN-TOKENS for similarity

# simi_PATH = '/home/mca_fix/share/bert-base-nli-mean-tokens/'
simi_PATH = '/home/mcafixmlpython/lib/bert-base-nli-mean-tokens/'

simi_tokenizer=AutoTokenizer.from_pretrained(simi_PATH)
simi_model=AutoModel.from_pretrained(simi_PATH,local_files_only=True)

import warnings
warnings.filterwarnings('ignore')

/home/mcafixmlpython/.local/lib/python3.6/site-packages/spacy/util.py:833: UserWarning: [W095] Model 'en_core_web_md' (3.0.0) was trained with spaCy v3.0 and may not be 100% compatible with the current version (3.2.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
Some weights of the model checkpoint at /home/mcafixmlpython/lib/bert-base-nli-mean-tokens/ were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a

CPU times: user 8.66 s, sys: 2.9 s, total: 11.6 s
Wall time: 9.45 s


In [2]:
%%time

def similar(sent):
    tokens={'input_ids':[],'attention_mask':[]}
    for sentence in sent:
        new_tokens=simi_tokenizer.encode_plus(sentence,max_length=128,truncation=True,padding='max_length',return_tensors='pt')
        tokens['input_ids'].append(new_tokens['input_ids'][0])
        tokens['attention_mask'].append(new_tokens['attention_mask'][0])
    #reformat list of tensors into single tensor
    tokens['input_ids']=torch.stack(tokens['input_ids'])
    tokens['attention_mask']=torch.stack(tokens['attention_mask'])
    #processing tokens
    outputs=simi_model(**tokens)
    outputs.keys()
    embeddings=outputs.last_hidden_state
    attention_mask=tokens['attention_mask']
    mask=attention_mask.unsqueeze(-1).expand(embeddings.size()).float()
    masked_embeddings=embeddings * mask
    summed = torch.sum(masked_embeddings,1)
    summed_mask=torch.clamp(mask.sum(1),min=1e-9)
    mean_pooled=summed/summed_mask
    from sklearn.metrics.pairwise import cosine_similarity
    mean_pooled=mean_pooled.detach().numpy()
    x=cosine_similarity([mean_pooled[0]],mean_pooled[1:])
    return x[0][0]

CPU times: user 3 µs, sys: 2 µs, total: 5 µs
Wall time: 8.58 µs


In [3]:
from nltk.corpus import stopwords

def remove_stopwords(input_text):
    #print("in remove_stopwords\n",input_text)
       
    stopwords_list = stopwords.words('english')
    newStopWords = ['citi']
    stopwords_list.extend(newStopWords)
        
    # Some words which might indicate a certain sentiment are kept via a whitelist
    #whitelist = ["n't", "not", "no"]
      
    whitelist = ["n't"]
    words = input_text.split() 
    clean_words = [word for word in words if (word not in stopwords_list or word in whitelist) and len(word) > 2]          
    return " ".join(clean_words)

### `1000 ARCMs.xlsx`

In [4]:
%%time 

df = pd.read_excel(io='1000 ARCMs.xlsx', sheet_name="1000 ARCMs", engine='openpyxl')

# df = df[df["Activity Description contains > 20 words"] == "Pass"]
df = df[["Activity Instance Id (for AU)", "Activity AU Description", "Frequency of the activity"]]
df.reset_index(drop=True, inplace=True)

df.dropna(axis=0, inplace=True)

# # coerce type conversion to every element of a DataFrame
# df = df.applymap(str)

# text preprocessing
# df['Activity AU Description'] = df['Activity AU Description'].str.strip().str.lower()
# df['Activity AU Description'] = df['Activity AU Description'].str.replace('[^\w\s]','') # semi-annual -> semiannual

print(df.shape)
df.head()

(1000, 3)
CPU times: user 2.52 s, sys: 33.8 ms, total: 2.56 s
Wall time: 2.56 s


,Activity Instance Id (for AU),Activity AU Description,Frequency of the activity
0,709382.0,Model Development Data Sourcing: On a daily (e...,daily basis
1,709382.0,Model Development Data Sourcing: On a daily (e...,daily basis
2,709382.0,Model Development Data Sourcing: On a daily (e...,daily basis
3,709382.0,Model Development Data Sourcing: On a daily (e...,daily basis
4,709383.0,Model Development Data Sourcing confidential P...,daily basis


In [5]:
when_frequency_lst = ["daily", "weekly", 
#                       "biweekly", "realtime", "event driven", # *** from control#
                      "monthly", "quarterly", "yearly", "semiannual", "semiannually",
                      "annual", # *** from activity# "ongoing", "needed", 
                      "annually", 
                      # outliers # 
                      "each month", "every month", "calendar month",  
                      # "needed basis", "required basis",
                      "ad hoc", "adhoc"]

from typing import Tuple

def activity_when(desc: str) -> Tuple[str, str, int, int]:
    """
    
    
    """
    
    # Text preprocessing
    desc = str(desc)
#     desc = re.sub('[^\w\s]', '', desc.strip().lower())
    desc = desc.lower()
    doc = nlp(desc)
    
    activity_when_answer = None
    
    
    ########################### semi/bi, semi-annually, bi-weekly... ###########################
    semi_bi = ["semi", "bi"]
    for word in when_frequency_lst:
        
        for semi_bi_word in semi_bi:
            
            if f"""{semi_bi_word}-{word}""" in desc:
        
                activity_when_answer = f"""{semi_bi_word}-{word}"""
        
    
    
    ########################### annual and quarterly basis; adhoc or weekly basis... ########################### 
    for token in doc:
        
        # Exclude last three tokens 
        try:
        
            if (token.text in when_frequency_lst) and (doc[token.i+1].pos_ == "CCONJ") and (doc[token.i+2].text in when_frequency_lst) and (doc[token.i+3].text in ["basis", "cadence"]):
                
                span = " ".join([doc[token.i].text, doc[token.i+1].text, doc[token.i+2].text, doc[token.i+3].text])

                if any(word in when_frequency_lst for word in span.split()) and (activity_when_answer is None):
  
                    activity_when_answer = span
        
        except:
            
            pass
    
    
    
    ########################### on ... basis... ########################### 
    for token in doc:
        
        # Exclude the last token 
        try:
            
            if (token.text in when_frequency_lst) and (doc[token.i+1].text in ["basis", "cadence"]):
                
                # ... as ... basis (monthly as needed basis)
                if doc[token.i-2].text in when_frequency_lst:
                    
                    span = " ".join([doc[token.i-2].text, doc[token.i-1].text, token.text, doc[token.i+1].text])
                
                # on a/an ... basis ..
                elif (doc[token.i-2].text == "on") and ((doc[token.i-1].text == "a") or (doc[token.i-1].text == "an")):
                    
                    span = " ".join([doc[token.i-2].text, doc[token.i-1].text, token.text, doc[token.i+1].text])
                
                # on ... basis ...
                elif doc[token.i-1].text == "on":
                    
                    span = " ".join([doc[token.i-1].text, token.text, doc[token.i+1].text])
                
                else:
                    
                    span = " ".join([token.text, doc[token.i+1].text])

                if any(word in when_frequency_lst for word in span.split()) and (activity_when_answer is None):

                        activity_when_answer = span
        
        except:
            
            pass
            
              
    ########################### monthly, quarterly, annually, semi-annually... ###########################       
    for token in doc:
        
        # Exclude the last token 
        try:
        
            if ((token.dep_ == "amod") and (token.pos_ == "ADJ")) or ((token.dep_ == "advmod") and (token.pos_ == "ADV")):
                
                span = token.text

                if any(word in when_frequency_lst for word in span.split()) and (activity_when_answer is None):

                    activity_when_answer = span
        
        except:
            
            pass
                
                
                
    ########################### other outliers... ###########################
    if activity_when_answer is None:
    
        for word in when_frequency_lst:
            
            if word in desc:
                
                activity_when_answer = word
    
    
    
    ########################### confidence ###########################
    if activity_when_answer is None:
        
        confidence = "Low"
        
    else:
        
        confidence = "High"
        
        
        
    ########################### starting index, ending index ###########################
    if activity_when_answer is None:
        
        start, end = None, None
    
    else:
        
        start = desc.find(activity_when_answer)
        
        if start == -1:
            
            start, end = None, None
        
        else:
            
            end = start + len(activity_when_answer)
        
    
    
    return activity_when_answer, confidence, start, end

In [6]:
%%time

# df["when_ans_freq"] = df["Activity AU Description"].apply(func=activity_when)
df[["when_ans_freq", "confidence", "start", "end"]] = df["Activity AU Description"].\
    apply(func=lambda row: activity_when(row)[0:4]).to_list()
df["found"] = (~df["when_ans_freq"].isna())
df['review_priority'] = np.where((df['found'] == False), "High Priority Review because of Missing When Frequency", "No Need to Review")      
df['missing_or_not'] = np.where((df['found'] == False), "Missing", "Not Missing")

CPU times: user 31.8 s, sys: 2.02 ms, total: 31.8 s
Wall time: 31.8 s


In [7]:
df[["Frequency of the activity", "when_ans_freq"]]

,Frequency of the activity,when_ans_freq
0,daily basis,daily
1,daily basis,daily
2,daily basis,daily
3,daily basis,daily
4,daily basis,on an daily basis
...,...,...
995,missing,None
996,missing,None
997,missing,None
998,missing,None


In [8]:
df[df["found"] == False]

,Activity Instance Id (for AU),Activity AU Description,Frequency of the activity,when_ans_freq,confidence,start,end,found,review_priority,missing_or_not
108,832522.0,CBORC Control - Onboarding: Onboarding is defi...,missing,None,Low,NaN,NaN,False,High Priority Review because of Missing When F...,Missing
160,894389.0,The Branded Cards Merchant Return Authorizatio...,missing,None,Low,NaN,NaN,False,High Priority Review because of Missing When F...,Missing
171,912722.0,Portfolio Quality Review. On a twice per quart...,twice per quarter cadence,None,Low,NaN,NaN,False,High Priority Review because of Missing When F...,Missing
182,925921.0,Disputed Tradeline Information in Suppression ...,missing,None,Low,NaN,NaN,False,High Priority Review because of Missing When F...,Missing
183,927041.0,Minimum Payment Due (MPD) – Branded Cards Risk...,missing,None,Low,NaN,NaN,False,High Priority Review because of Missing When F...,Missing
...,...,...,...,...,...,...,...,...,...,...
994,700454.0,(AU 504918 A 552962) ID_129973 Process Instanc...,missing,None,Low,NaN,NaN,False,High Priority Review because of Missing When F...,Missing
995,700456.0,(AU 504918 A 552911) ID_41450 Process Instance...,missing,None,Low,NaN,NaN,False,High Priority Review because of Missing When F...,Missing
996,702746.0,(AU 504918 A 552904) ID_177401 Process Instanc...,missing,None,Low,NaN,NaN,False,High Priority Review because of Missing When F...,Missing
997,702747.0,(AU 504918 A 552909) ID_177402 Process Instanc...,missing,None,Low,NaN,NaN,False,High Priority Review because of Missing When F...,Missing


### `similar()`

In [9]:
similar(sent=["monthly basis", "on a monthly basis"])

0.9803053

In [10]:
similar(sent=["semi-annual", "on a semiannual basis"])

0.85882294

In [11]:
df["when_ans_freq"] = df["when_ans_freq"].apply(func=lambda row : "missing" if row is None else row)

In [12]:
%%time

df["Frequency of the activity"] = df["Frequency of the activity"].str.strip().str.lower()
df["when_ans_freq"] = df["when_ans_freq"].str.strip().str.lower()
df["similarity"] = df.apply(func=lambda row: similar(sent=[row["Frequency of the activity"], row["when_ans_freq"]]), 
                            axis=1)
df["Frequency of the activity"] = df["Frequency of the activity"].apply(func=remove_stopwords)
df["when_ans_freq"] = df["when_ans_freq"].apply(func=remove_stopwords)

CPU times: user 20min 30s, sys: 41.8 s, total: 21min 12s
Wall time: 3min 37s


# `Conclusion:`

### `True Postive: 882 records`

In [13]:
tp = df[ (df["Frequency of the activity"] != "missing") & (df["when_ans_freq"] != "missing") ]
tp = tp[["Frequency of the activity", "when_ans_freq", "similarity"]]
tp = tp.loc[tp["similarity"] >= 0.80]
print(tp.shape)
tp

(881, 3)


,Frequency of the activity,when_ans_freq,similarity
0,daily basis,daily,0.984745
1,daily basis,daily,0.984745
2,daily basis,daily,0.984745
3,daily basis,daily,0.984745
4,daily basis,daily basis,0.976222
...,...,...,...
984,quarterly basis,quarterly basis,0.982251
985,daily basis,daily basis,0.974561
989,daily basis,daily basis,0.974561
990,daily basis,daily basis,0.974561


### `True Negative: 107 records`

In [14]:
tn = df[(df["Frequency of the activity"] == "missing") & (df["when_ans_freq"] == "missing") ]
tn = tn[["Frequency of the activity", "when_ans_freq", "similarity"]]
print(tn.shape)
tn

(107, 3)


,Frequency of the activity,when_ans_freq,similarity
108,missing,missing,1.0
160,missing,missing,1.0
182,missing,missing,1.0
183,missing,missing,1.0
184,missing,missing,1.0
...,...,...,...
994,missing,missing,1.0
995,missing,missing,1.0
996,missing,missing,1.0
997,missing,missing,1.0


### `False Positive: 6 records`

In [15]:
fp = df[(df["similarity"] < 0.80) & (df["when_ans_freq"] != "missing")].loc[:, ["Activity Instance Id (for AU)", "Frequency of the activity", "when_ans_freq", "similarity"]]
print(fp.shape)
fp

(7, 4)


,Activity Instance Id (for AU),Frequency of the activity,when_ans_freq,similarity
693,945808.0,missing,daily,0.288967
694,945808.0,missing,daily,0.288967
752,1013621.0,adhoc basis,monthly,0.432944
753,1013621.0,adhoc basis,monthly,0.432944
754,1013621.0,adhoc basis,monthly,0.432944
755,1013621.0,adhoc basis,monthly,0.432944
756,1013621.0,adhoc basis,monthly,0.432944


In [25]:
df.loc[752, "Activity AU Description"]

'Communication Change Management. On a monthly (ad-hoc) basis, the Pricing Change Management Team governs the process of a customer communication change that is requested by the Communication Change Owner (CCO) via the Communication Change Memo (Memo) on behalf of U.S. Branded Cards for all customer touch points. This Activity is executed to inform potentially impacted Channel Partners to ensure accurate and timely communication of product, benefit/feature, regulatory, compliance and/or legal changes. Dependencies: Applications/Systems: CSI # 153591 - SharePoint Global Services a.k.a. WFM Communication Change Management (CCM) tool.'

### `False Negative: 5 records`

In [17]:
fn = df[(df["Frequency of the activity"] != "missing") & 
         (df["when_ans_freq"] == "missing") ]
fn = fn[["Activity Instance Id (for AU)", "Frequency of the activity", "when_ans_freq", "similarity"]]
print(fn.shape)
fn

(5, 4)


,Activity Instance Id (for AU),Frequency of the activity,when_ans_freq,similarity
171,912722.0,twice per quarter cadence,missing,0.245766
214,942006.0,ongoing basis,missing,0.414187
215,942006.0,ongoing basis,missing,0.414187
216,942006.0,ongoing basis,missing,0.414187
318,886500.0,ongoing,missing,0.469507


### `Accuracy`

In [18]:
accuracy = (len(tp) + len(tn)) / (len(tp) + len(fp) + len(fn) + len(tn))
accuracy

0.988

### `Precision`

In [19]:
precision = len(tp) /  (len(tp) + len(fp))
precision

0.9921171171171171

### `Recall`

In [20]:
recall = len(tp) /  (len(tp) + len(fn))
recall

0.9943566591422122

### `F1-Score`

In [21]:
f1_score = 2 * precision * recall / (precision + recall)
f1_score

0.9932356257046222

### `Confusion Matrix`

In [22]:
cm = pd.DataFrame(columns=["Postive", "Negative"], index=["Postive", "Negative"])
cm["Postive"], cm["Negative"] = [len(tp), len(fn)], [len(fp), len(tn)]
cm

,Postive,Negative
Postive,881,7
Negative,5,107


In [23]:
cm.sum().sum()

1000

In [24]:
len(df)

1000